# Making a function for the decoding
Testing on a small dataset


#### Code used for batches below 

In [ ]:
# !pip install xdot --user

In [1]:
import numpy as np
import nilearn
import nibabel as nib
import pandas as pd 
from sklearn.model_selection import cross_val_score
from nilearn.input_data import NiftiMasker 
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Agg')
from sklearn.svm import SVC
from sklearn.feature_selection import SelectPercentile, f_classif, SelectKBest
from sklearn.pipeline import Pipeline

from beaker.cache import CacheManager
from beaker.util import parse_cache_config_options
import objgraph
# import xdot
import pdb

import multiprocessing
from multiprocessing import Pool

import time
from functools import partial

/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec_

## Cache Money

In [2]:
cache_opts = {
    'cache.type': 'file',
    'cache.data_dir': '/Users/gracer/Google Drive/decode/tmp/data',
    'cache.lock_dir': '/Users/gracer/Google Drive/decode/tmp/lock',
    'cache.regions' : 'short_term, long_term'
}

cache = CacheManager(**parse_cache_config_options(cache_opts))

tmpl_cache = cache.get_cache('/Users/gracer/Google Drive/decode/tmp/mytemplate.html', type='dbm', expire=3600)

will want to cache the following:
* masker
* the fit (X)
* anova_svc
* y_pred


## Fitter function 

In [19]:
@cache.cache('short_term')
def fitter(IMAG_MASK, DATASET, LABELS ,CONDITION_MASK, ANOVA_SVC, CONDITIONS):
    masker = NiftiMasker(mask_img=IMAG_MASK, standardize=True, memory="nilearn_cache", memory_level=1)
    X = masker.fit_transform(DATASET.func[0])
    
    # Apply our condition_mask
    X = X[CONDITION_MASK]

    
#     ANOVA_SVC.fit(X,CONDITIONS)
#     y_pred = ANOVA_SVC.predict(X)
    
    return([ANOVA_SVC.fit(X,CONDITIONS),X])


In [20]:
# NESTED CROSS VALIDATION

def run_CV(MODEL, PARAMS):
    grid = GridSearchCV(MODEL, param_grid={'anova__k': PARAMS}, verbose=1, cv=5, n_jobs=4)
    return cross_val_score(grid, X, y, cv=5)
        


In [28]:
haxby_dataset = nilearn.datasets.fetch_haxby()
ANOVA_SVC=test(haxby_dataset, haxby_dataset.mask, pd.read_csv(haxby_dataset.session_target[0], sep=" "))
    
from sklearn.model_selection import GridSearchCV
k_range = [[ 15, 50, 150, 300], [500, 1000, 3000, 5000]]
ANOVA_SVC

['face' 'house']


Pipeline(memory=None,
     steps=[('anova', SelectKBest(k=500, score_func=<function f_classif at 0x1c1059c268>)), ('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [30]:
def run_CV(params):
    grid = GridSearchCV(ANOVA_SVC, param_grid={'anova__k': params}, verbose=1, cv=5, n_jobs=4)
    return cross_val_score(grid, X, y, cv=5)
        
  
        

p = Pool(processes = 4) 
start = time.time()
nested_cv_scores =  p.map(run_CV, k_range)
p.close()
p.join()
print("Nested CV score: %.4f" % np.mean(nested_cv_scores))
print("Time: ", (time.time() - start))

NameError: name 'X' is not defined

In [21]:
run_CV(test(haxby_dataset, haxby_dataset.mask, pd.read_csv(haxby_dataset.session_target[0], sep=" ")), k_range)

['face' 'house']


NameError: name 'X' is not defined

In [26]:
def test(DATASET, MASK, BEHAVIORAL):
    from sklearn.svm import SVC
    from sklearn.feature_selection import SelectPercentile, f_classif, SelectKBest
    from sklearn.pipeline import Pipeline

    svc = SVC(kernel='linear')
    FEATURE_SELECTION = SelectKBest(f_classif, k=500)
    ANOVA_SVC = Pipeline([('anova', FEATURE_SELECTION), ('svc', svc)])
    
    #image mask
    IMAG_MASK=MASK

    BEHAVIORAL = BEHAVIORAL
    CONDITIONS = BEHAVIORAL['labels']

    #our dataset concatenated image 
    DATASET=DATASET

    LABELS = BEHAVIORAL["labels"]
    #restrict data to our target analysis 
    CONDITION_MASK = BEHAVIORAL['labels'].isin(['face', 'house'])
    CONDITIONS = CONDITIONS[CONDITION_MASK]

    #confirm we have the # of condtions needed
    print(CONDITIONS.unique())
    [ANOVA_SVC_FIT,X]=fitter(IMAG_MASK, DATASET, LABELS ,CONDITION_MASK, ANOVA_SVC, CONDITIONS)
    return(ANOVA_SVC_FIT)
    
    
    
    

In [12]:

def parallel_runs(data_list):
    pool = multiprocessing.Pool(processes=4)
    ANOVA_SVC_FIT = test(haxby_dataset, haxby_dataset.mask, pd.read_csv(haxby_dataset.session_target[0], sep=" "))
    pdb.set_trace()
    
    prod_x=partial(run_CV, MODEL=ANOVA_SVC_FIT) 
    result_list = pool.map(prod_x, data_list) 
    print(result_list)



In [14]:
if __name__ == '__main__':
    start = time.time()
    parallel_runs(k_range)
    print("Nested CV score: %.4f" % np.mean(nested_cv_scores))
    print("Time: ", (time.time() - start))

['face' 'house']
> <ipython-input-12-8ccaf5747bd2>(6)parallel_runs()
-> prod_x=partial(run_CV, MODEL=ANOVA_SVC_FIT)
(Pdb) ANOVA_SVC_FIT
Pipeline(memory=None,
     steps=[('anova', SelectKBest(k=500, score_func=<function f_classif at 0x1c1059c268>)), ('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
(Pdb) c


TypeError: run_CV() got multiple values for argument 'MODEL'

## Testing 1, 2

In [ ]:
import pandas as pd

# Load target information as string and give a numerical identifier to each
behavioral = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
conditions = behavioral['labels']

# Restrict the analysis to faces and places
condition_mask = behavioral['labels'].isin(['face', 'house'])
conditions = conditions[condition_mask]

# Confirm that we now have 2 conditions
print(conditions.unique())

# Record these as an array of sessions, with fields
# for condition (face or house) and run
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)

In [ ]:
haxby_dataset = nilearn.datasets.fetch_haxby()
main(haxby_dataset, haxby_dataset.mask, pd.read_csv(haxby_dataset.session_target[0], sep=" "))

In [ ]:

# masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
# X = masker.fit_transform(dataset)
# # Apply our condition_mask
# X = X[condition_mask]



# # PREDICTION FUNCTION
# svc = SVC(kernel='linear')

# # FEATURE SELECTION
# feature_selection = SelectKBest(f_classif, k=500)


# anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])
# anova_svc.fit(X,y)
# y_pred = anova_svc.predict(X)

# # NESTED CROSS VALIDATION 
# nested_cv_scores = cross_val_score(anova_svc, X, y,  cv=5) 
# #cv_scores = cross_val_score(anova_svc, X, conditions,)
# # Print the results
# print("Nested CV score: %.4f" % np.mean(nested_cv_scores))


# # In[ ]:


# # Here is the image 
# coef = svc.coef_
# # reverse feature selection
# coef = feature_selection.inverse_transform(coef)
# # reverse masking
# weight_img = masker.inverse_transform(coef)


# # Use the mean image as a background to avoid relying on anatomical data
# from nilearn import image
# mean_img = image.mean_img(dataset)
# mean_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/all/all_waves_b%s_mean_nimask.nii'%batchID)

# # Create the figure
# from nilearn.plotting import plot_stat_map, show
# display = plot_stat_map(weight_img, mean_img, title='Milkshake vs. h2O')
# display.savefig('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/all/all_waves_b%s_SVM_nimask.png'%batchID)
# # Saving the results as a Nifti file may also be important
# weight_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/milkshake_vs_h2O/images/all/all_waves_b%s_SVM_nimask.nii'%batchID)



In [ ]:
from nilearn import datasets

# By default 2nd subject will be fetched
haxby_dataset = datasets.fetch_haxby()

# print basic information on the dataset
print('Mask nifti image (3D) is located at: %s' % haxby_dataset.mask)
print('Functional nifti image (4D) is located at: %s' %
      haxby_dataset.func[0])

In [ ]:
import pandas as pd

# Load target information as string and give a numerical identifier to each
behavioral = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
conditions = behavioral['labels']

# Restrict the analysis to faces and places
condition_mask = behavioral['labels'].isin(['face', 'house'])
conditions = conditions[condition_mask]

# Confirm that we now have 2 conditions
print(conditions.unique())

# Record these as an array of sessions, with fields
# for condition (face or house) and run
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)

In [ ]:
from nilearn.input_data import NiftiMasker

mask_filename = haxby_dataset.mask

# For decoding, standardizing is often very important
# note that we are also smoothing the data
masker = NiftiMasker(mask_img=mask_filename, smoothing_fwhm=4,
                     standardize=True, memory="nilearn_cache", memory_level=1)
func_filename = haxby_dataset.func[0]
X = masker.fit_transform(func_filename)
# Apply our condition_mask
X = X[condition_mask]

In [ ]:
from nilearn import image
MEAN_IMG = image.mean_img(haxby_dataset)

In [ ]:
haxby_dataset